In [117]:
from tree_sitter import Language, Parser  # 解析器库
import re
import pandas as pd
import Levenshtein
import os
import sys
import numpy as np




In [118]:
data = pd.read_excel("./dataset3/train/train_all.xlsx")
# data = pd.read_csv("./dataset/megavul_simple_cpp_success_getast.csv")
# data = pd.read_excel("./dataset/megavul_simple_cpp_all.xlsx")

In [119]:
data

,cve_id,func_before,description,Base Score,Base Severity
0,CVE-2023-38850,"static void write_html_body( FILE *out,...",Buffer Overflow vulnerability in Michaelrsweet...,5.5,MEDIUM
1,CVE-2017-15587,static void pdf_read_new_xref_section(fz_conte...,An integer overflow was discovered in pdf_read...,7.8,HIGH
2,CVE-2023-36364,"static int rel_deps(mvc *sql, sql_rel *r, list...",An issue in the rel_deps component of MonetDB ...,7.5,HIGH
3,CVE-2019-19319,int ext4_setup_system_zone(struct super_block ...,"In the Linux kernel before 5.2, a setxattr ope...",6.5,MEDIUM
4,CVE-2019-19948,static MagickBooleanType WriteSGIImage(const I...,"In ImageMagick 7.0.8-43 Q16, there is a heap-b...",9.8,CRITICAL
...,...,...,...,...,...
9651,CVE-2019-12448,static void do_query_info (GVfsBackend *backen...,An issue was discovered in GNOME gvfs 1.29.4 t...,8.1,HIGH
9652,CVE-2018-10853,static int linear_read_system(struct x86_emula...,A flaw was found in the way Linux kernel KVM h...,7.0,HIGH
9653,CVE-2020-27670,static unsigned int clear_iommu_pte_present(un...,An issue was discovered in Xen through 4.14.x ...,7.8,HIGH
9654,CVE-2019-19082,struct resource_pool *dce100_create_resource_p...,Memory leaks in *create_resource_pool() functi...,4.7,MEDIUM


In [120]:
code = data['func_before']

In [121]:
code

0       static void write_html_body( FILE        *out,...
1       static void pdf_read_new_xref_section(fz_conte...
2       static int rel_deps(mvc *sql, sql_rel *r, list...
3       int ext4_setup_system_zone(struct super_block ...
4       static MagickBooleanType WriteSGIImage(const I...
                              ...                        
9651    static void do_query_info (GVfsBackend *backen...
9652    static int linear_read_system(struct x86_emula...
9653    static unsigned int clear_iommu_pte_present(un...
9654    struct resource_pool *dce100_create_resource_p...
9655    static void inputSendThreadProc(void* context)...
Name: func_before, Length: 9656, dtype: object

In [122]:
code[0]

'static void write_html_body( FILE        *out, int         mode, const char  *bodyfile, mmd_t       *body, mxml_node_t *doc) { mxml_node_t*function, *scut, *arg, *description, *type; const char*name, *defval; boolwhitespace; const char*string; if (body) markdown_write_block(out, body, mode); else if (bodyfile) write_file(out, bodyfile, mode); if ((scut = find_public(doc, doc, "class", NULL, mode)) != NULL) { fputs("<h2 class=\\"title\\"><a id=\\"CLASSES\\">Classes</a></h2>\\n", out); while (scut) { write_scu(out, mode, doc, scut); scut = find_public(scut, doc, "class", NULL, mode); } } if ((function = find_public(doc, doc, "function", NULL, mode)) != NULL) { fputs("<h2 class=\\"title\\"><a id=\\"FUNCTIONS\\">Functions</a></h2>\\n", out); while (function) { write_function(out, mode, doc, function, 3); function = find_public(function, doc, "function", NULL, mode); } } if ((scut = find_public(doc, doc, "typedef", NULL, mode)) != NULL) { fputs("<h2 class=\\"title\\"><a id=\\"TYPES\\">Data

In [123]:
ast = []

In [124]:
# idx = []

In [125]:
CPP_LANGUAGE = Language('build/my-languages.so', 'cpp')
cpp_parser = Parser()
cpp_parser.set_language(CPP_LANGUAGE)

C:\APP\Anaconda3\envs\funcGNN_gcy\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [126]:
for i in range(len(code)):
    tree = cpp_parser.parse(bytes(code[i], "utf8"))
    root = tree.root_node  # 注意，root_node 才是可遍历的树节点
    sexp = root.sexp()
    cleaned_sexp = re.sub(r'[:\(\)]', '', sexp)
    ast.append(cleaned_sexp)
    # ast.append(sexp)

In [127]:
print(len(ast))

9656


In [128]:
print(ast[0][:100])

translation_unit function_definition storage_class_specifier type primitive_type declarator function


In [129]:
# for i in range(len(ast)):
#     if ast[i][:300].find("ERROR") != -1:
#         idx.append(i)

In [130]:
# idx

In [131]:
# print(len(idx))

In [132]:
# data2 = data.drop(labels=idx,axis=0)

In [133]:
# data2

In [134]:
# print(len(data2))

In [135]:
# data2.to_csv("./dataset/megavul_simple_cpp_success_ast.csv", index=False)

In [136]:


print(Levenshtein.seqratio(ast[0].split(), ast[1].split()))

0.35547212968085523


In [137]:
df = pd.DataFrame(ast)

In [138]:
print(len(df))

9656


In [139]:
df.to_csv("./dataset3/train/train_ast.csv", header=False, index=False)

In [25]:
ddd = pd.read_csv("./dataset2/test/test_ast.csv", header=None)

In [26]:
ddd

,0
0,translation_unit function_definition type type...
1,translation_unit function_definition storage_c...
2,translation_unit function_definition type prim...
3,translation_unit function_definition storage_c...
4,translation_unit function_definition storage_c...
...,...
1203,translation_unit function_definition storage_c...
1204,translation_unit function_definition type type...
1205,translation_unit function_definition type prim...
1206,translation_unit function_definition storage_c...


In [86]:
import csv
chat = pd.read_csv("./dataset/valid_ast.csv",sep = "\t", header = None)

In [87]:
chat

,0
0,translation_unit function_definition storage_c...
1,translation_unit function_definition storage_c...
2,translation_unit function_definition storage_c...
3,translation_unit function_definition type prim...
4,translation_unit function_definition type temp...
...,...
1418,translation_unit function_definition type prim...
1419,translation_unit function_definition type stru...
1420,translation_unit function_definition storage_c...
1421,translation_unit function_definition storage_c...
